In [1]:
import torch
import numpy as np
import torch.nn as nn
import math


'''MLP AND LINEARS'''

def init_relu(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.orthogonal_(module.weight, gain=1.41421)

        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_orth(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.orthogonal_(module.weight, gain=1)

        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_xavier(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.xavier_uniform_(module.weight, gain=1)

        if module.bias is not None:
            nn.init.zeros_(module.bias)
            
def init_xavier_normal(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.xavier_normal_(module.weight, gain=1)

        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_zeros(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.zeros_(module.weight)

        if module.bias is not None:
            nn.init.zeros_(module.bias)


def init_sigmoid(module):
    #print(f"The init sigmoid was only tested by the package's author at the CfC.")
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.xavier_normal_(module.weight, gain=1)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_lecun(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.normal_(module.weight, mean=0.0, std=1.0 / (module.weight.shape[1])**0.5)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_tanh(module):
    #print(f"The init tanh was only tested by the package's author at the CfC.")
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.xavier_normal_(module.weight, gain=1.6667)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_deep_lstm(module):
    # Ref: Sequence to Sequence Learning with Neural Networks
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        nn.init.uniform_(module.weight, -0.08, 0.08)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_alphastar_special(module):
    # Ref: Alphastar
    if isinstance(module, nn.Linear):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.005)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)

def init_emb(module):
    if type(module) == nn.Linear:
        torch.nn.init.normal_(module.weight, std=math.sqrt(1/module.weight.shape[0]))
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias) 
            
    if type(module) == nn.Embedding:
        torch.nn.init.normal_(module.weight, std=math.sqrt(1/module.weight.shape[1]))

def init_saving_variance(module, num_blks):
    
    torch.nn.init.xavier_uniform_(module.weight, gain=torch.tensor(4*num_blks).pow(-1/4))
    if hasattr(module, 'bias'):
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
            

def init_gpt(module):
    #print(f"From init_gpt.\nGpt proj linears should have a special weight initialization not implemented here.")
    if isinstance(module, nn.Linear):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        #torch.nn.init.xavier_normal_(module.weight)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        #torch.nn.init.xavier_normal_(module.weight)
    elif isinstance(module, nn.LayerNorm):
        nn.init.constant_(module.bias, 0)
        nn.init.constant_(module.weight, 1.0)
        

def init_proj(module):
    assert not isinstance(module, nn.Conv1d) and not isinstance(module, nn.Conv2d) and not isinstance(module, nn.Conv3d)
    if isinstance(module, nn.Linear):
        nn.init.eye_(module.weight)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)




        
'''CNN'''

def init_cnn(module):
    if type(module) == nn.Linear or type(module) == nn.Conv2d or type(module) == nn.Conv1d or type(module) == nn.Conv3d:
        #nn.init.kaiming_uniform_(module.weight, a=0, mode='fan_in', nonlinearity='SiLU')
        nn.init.orthogonal_(module.weight, 1)
        #nn.init.orthogonal_(module.weight, 1.41421)
        #nn.init.xavier_uniform_(module.weight, 1)
        #nn.init.xavier_uniform_(module.weight, 1.41421)


        if module.bias is not None:
            nn.init.zeros_(module.bias)

def init_partial_dirac(module):
    if type(module) in (nn.Conv2d, nn.Conv1d, nn.Conv3d):
        w = module.weight.data
        
        nn.init.dirac_(module.weight[:w.shape[1]])
        nn.init.xavier_uniform_(module.weight[w.shape[1]:], gain=1)

        if module.bias is not None:
            nn.init.zeros_(module.bias)
    if type(module) == nn.Linear:
        print(f"ERROR: ONLY CONVOLUTIONS ARE SUPPORTED BY THE DIRAC INITIALIZATION.")

def init_dreamer_normal(module):
    if type(module) == nn.Linear or type(module) == nn.Conv2d or type(module) == nn.Conv1d or type(module) == nn.Conv3d:

        if type(module)==nn.Linear():
            space = module.weight.shape[1] * module.weight.shape[0]
            in_num = space * module.weight.shape[1]
            out_num = space * module.weight.shape[1]
        else:
            space = module.kernel_size[0] * module.kernel_size[1]
            in_num = space * module.in_channels
            out_num = space * module.out_channels
        
        std = np.sqrt((1/np.mean(np.array([in_num, out_num])))) / 0.87962566103423978
        nn.init.trunc_normal_(module.weight.data, mean=0.0, std=std, a=-2.0 * std, b=2.0 * std)
        

        if module.bias is not None:
            nn.init.zeros_(module.bias)
        

def init_dreamer_uniform(m):
    # Same as xavier uniform
    '''
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        #nn.init.orthogonal_(m.weight, 1.41421)
    '''
    if isinstance(m, nn.Linear):
        in_num = m.in_features
        out_num = m.out_features
        denoms = (in_num + out_num) / 2.0
        scale = 1.0 / denoms
        limit = np.sqrt(3 * scale)
        nn.init.uniform_(m.weight.data, a=-limit, b=limit)
        if hasattr(m.bias, "data"):
            m.bias.data.fill_(0.0)
    

    
def init_proj2d(module):
    if type(module) in (nn.Linear, nn.Conv2d, nn.Conv1d, nn.Conv3d):
        torch.nn.init.dirac_(module.weight, groups=1)
        
        if module.bias is not None:
            nn.init.zeros_(module.bias)


'''WHITENED LAYERS'''

def get_patches(x, patch_shape):
    c, (h, w) = x.shape[1], patch_shape
    
    return x.unfold(2,h,1).unfold(3,w,1).transpose(1,3).reshape(-1,c,h,w).float()

def get_whitening_parameters(patches):
    n,c,h,w = patches.shape
    patches_flat = patches.view(n, -1)
    est_patch_covariance = (patches_flat.T @ patches_flat) / n
    
    eigenvalues, eigenvectors = torch.linalg.eigh(est_patch_covariance, UPLO='U')
    
    return eigenvalues.flip(0).view(-1, 1, 1, 1), eigenvectors.T.reshape(c*h*w,c,h,w).flip(0)

def init_whitening_conv(layer, train_set, eps=5e-4):
    patches = get_patches(train_set, patch_shape=layer.weight.data.shape[2:])
    
    eigenvalues, eigenvectors = get_whitening_parameters(patches)
    
    eigenvectors_scaled = eigenvectors / torch.sqrt(eigenvalues + eps)
    
    layer.weight.data[:] = torch.cat((eigenvectors_scaled, -eigenvectors_scaled))
    layer.weight.requires_grad=False

In [2]:
# REFERENCES
# https://github.com/karpathy/nanoGPT
# https://github.com/JegZheng/truncated-diffusion-probabilistic-models
# https://github.com/facebookresearch/DiT/blob/main/models.py

import torch
from torch import nn
import torch.nn.functional as F
import math



@torch.jit.script # JIT decorator
def fused_gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))
from torch import nn
import inspect
def network_ema(target_network, new_network, alpha=0.5):
    for (param_name, param_target), param_new  in zip(target_network.cuda().named_parameters(), new_network.parameters()):
        if 'ln' in param_name: #layer norm
            param_target.data = param_new.data.clone()
        else:
            param_target.data = alpha * param_target.data + (1 - alpha) * param_new.data.clone()
import torch
import torch.nn.functional as F
import numpy as np

import random
import os


def params_count(model, name='Model'):
    params_to_count = [p for p in model.parameters() if p.requires_grad]
    print(f'{name} Parameters: {sum(p.numel() for p in params_to_count)/1e6:.2f}M')


def params_and_grad_norm(model):
    param_norm, grad_norm = 0, 0
    for n, param in model.named_parameters():
        if not n.endswith('.bias'):
            param_norm += torch.norm(param.data)
            if param.grad is not None:
                grad_norm += torch.norm(param.grad)
    return param_norm, grad_norm


# From STORM Atari-100k
def seed_np_torch(seed=20001118):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    

def statistical_difference(p1, p2, n):
    # order invariant
    
    d=torch.tensor(p1-p2).abs()
    std = 1.65 * math.sqrt((p1*(1-p1) + p2*(1-p2))/n)
    difference = torch.tensor([d-std, d+std])
        
    difference = difference.sort()[0]
    
    return difference

def renormalize(tensor):
    shape = tensor.shape
    tensor = tensor.view(shape[0], -1)
    max_value,_ = torch.max(tensor, -1, keepdim=True)
    min_value,_ = torch.min(tensor, -1, keepdim=True)
    return ((tensor - min_value) / (max_value - min_value + 1e-5)).view(shape)

# Hyper Parameters
# automatically saves all arguments of the inherited class __init__
class Hypers: # Sorcery
    def __init__(self, max_depth=3, **kwargs):
        super().__init__(**kwargs)
        self.save_hypers(max_depth)
    
    def save_hypers(self, max_depth, ignore=[]):
      """Save function arguments into class attributes."""

      #f_back: frame caller
      #frame: table of local variablies to the frame's function
      seen_init=False
      frame = inspect.currentframe()
      for d in range(max_depth):
          
          frame = frame.f_back
          
          if frame.f_back and frame.f_back.f_code.co_name == "__init__":
              seen_init=True
              
          if seen_init and frame.f_back.f_code.co_name != "__init__":
              break
            
      _, _, _, local_vars = inspect.getargvalues(frame)
      #takes the arguments of the function which called this save_hypers function
      #it can backtrack functions according to the depth argument

      self.hparams = {k:v for k, v in local_vars.items()
          if k not in set(ignore+['self']) and not k.startswith('_')}
      for k, v in self.hparams.items():
          setattr(self, k, v)


# ALLWAYS PUT HYPERS TO THE LEFT
class nsd_Module(Hypers, nn.Module):
    def __init__(self):
        super().__init__(max_depth=3)
class FusedGELU(nn.Module):
    def forward(self, x):
        return fused_gelu(x)


class LayerNormNoBias(nn.Module):
    """ LayerNormNoBias but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, d_model, bias=False):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(d_model))
        self.bias = nn.Parameter(torch.zeros(d_model)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)


    
class Attention(nsd_Module):
    def __init__(self, d_model=512, nhead=8, bias=False, dropout=0.1, seq_len=8):
        super().__init__()
        # key, query, value projections for all heads, but in a batch
        self.W_q = nn.Linear(d_model, d_model, bias=bias)
        self.W_k = nn.Linear(d_model, d_model, bias=bias)
        self.W_v = nn.Linear(d_model, d_model, bias=bias)
        # output projection
        self.proj = nn.Linear(d_model, d_model, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

        self.seq_len = seq_len
        self.k_pre = None
        self.k_post = None

    def forward(self, q, k, v, is_causal):
        B, T, C = q.size()
        
        q = self.W_k(q)
        k = self.W_k(k)
        v = self.W_v(v)
        
        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y

    def forward_xl(self, q, k, v, is_causal):
        B, T, C = q.size()

        q = self.W_k(q)
        k = self.W_k(k)
        v = self.W_v(v)

        self.k_pre = k.detach()
        self.v_pre = v.detach()
        if self.k_post!=None:
            k = torch.cat((self.post,k),-2)
            v = torch.cat((self.post,v),-2)
        
        self.k_post = self.k_pre
        self.v_post = self.v_pre

        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y
    
    def forward_xl_windowed(self, q, k, v, is_causal):
        B, T, C = q.size()
        
        q = self.W_k(q)
        k = self.W_k(k)
        v = self.W_v(v)

        if self.k_pre == None:
            self.k_pre = k.detach()
            self.v_pre = v.detach()
        elif self.k_pre.shape[-2] < self.seq_len:
            self.k_pre = k.detach()
            self.v_pre = v.detach()
        else:
            self.k_pre = k[...,1:,:].detach()
            self.v_pre = v[...,1:,:].detach()


        if self.k_post!=None:
            k = torch.cat((self.k_post,k),-2)
            v = torch.cat((self.v_post,v),-2)
        

        self.k_post = self.k_pre
        self.v_post = self.v_pre



        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y

    
class Attention_XL(nsd_Module):
    def __init__(self, d_model=512, nhead=8, bias=False, dropout=0.1):
        super().__init__()
        # key, query, value projections for all heads, but in a batch
        self.W_q = nn.Linear(d_model, d_model, bias=bias)
        self.W_k = nn.Linear(d_model, d_model, bias=bias)
        self.W_v = nn.Linear(d_model, d_model, bias=bias)
        # output projection
        self.proj = nn.Linear(d_model, d_model, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, is_causal):
        B, T, C = q.size()

        q = self.W_k(q)
        k = self.W_k(k)
        v = self.W_v(v)

        self.k_pre = k.detach()
        self.v_pre = v.detach()
        if self.k_post!=None:
            k = torch.cat((self.post,k),-2)
            v = torch.cat((self.post,v),-2)
        
        self.k_post = self.k_pre
        self.v_post = self.v_pre

        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y

    
class Attention_XL_window(nsd_Module):
    def __init__(self, d_model=512, nhead=8, bias=False, dropout=0.1, seq_len=8):
        super().__init__()
        # key, query, value projections for all heads, but in a batch
        self.W_q = nn.Linear(d_model, d_model, bias=bias)
        self.W_k = nn.Linear(d_model, d_model, bias=bias)
        self.W_v = nn.Linear(d_model, d_model, bias=bias)
        # output projection
        self.proj = nn.Linear(d_model, d_model, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.seq_len = seq_len

    def forward(self, q, k, v, is_causal):
        B, T, C = q.size()
        
        q = self.W_k(q)
        k = self.W_k(k)
        v = self.W_v(v)

        if self.k_pre == None:
            self.k_pre = k.detach()
            self.v_pre = v.detach()
        elif self.k_pre.shape[-2] < self.seq_len:
            self.k_pre = k.detach()
            self.v_pre = v.detach()
        else:
            self.k_pre = k[...,1:,:].detach()
            self.v_pre = v[...,1:,:].detach()


        if self.k_post!=None:
            k = torch.cat((self.post,k),-2)
            v = torch.cat((self.post,v),-2)
        

        self.k_post = self.k_pre
        self.v_post = self.v_pre



        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y



class MemoryAttention(nsd_Module):
    def __init__(self, d_model=512, nhead=8, bias=False, dropout=0.1):
        super().__init__()
        # key, query, value projections for all heads, but in a batch
        self.W_kv = nn.Linear(d_model, 2 * d_model, bias=bias)
        # output projection
        self.proj = nn.Linear(d_model, d_model, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

    def forward(self, x, q):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k, v  = self.W_kv(x).split(self.n_embd, dim=2)
        
        
        # FoT LongLlama contrastive style (data pipeline constrastive for self attention enrichment)
        
        shifted_k=[]
        shifted_v=[]
        for i in range(7): # 7 is d-1 for d=8
            shifted_k.append(torch.roll(k[:,:T//2],i,0))
            shifted_v.append(torch.roll(v[:,:T//2],i,0))
        shifted_k=torch.stack(shifted_k).view(B,-1,C)
        shifted_v=torch.stack(shifted_v).view(B,-1,C)
        
        k=torch.concat((shifted_k,k),1)
        v=torch.concat((shifted_v,v),1)
        
        
        
        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        
        
        
        L = q.shape[2]
        S = k.shape[2]
        attn_mask = torch.ones(L, S, dtype=torch.bool, device='cuda').tril(diagonal=S-L)
        attn_mask[:T//2,:S-L]=False
        
        
        
        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        
        # efficient attention using Flash Attention CUDA kernels
        with torch.backends.cuda.sdp_kernel():
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=False)
            #y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, dropout_p=self.dropout if self.training else 0)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.proj(y))
        return y

    def forward_memory(self, x, q, k_read, v_read):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        
        k, v = self.W_kv(x).split(self.n_embd, dim=2)
        write_k, write_v = k.detach(), v.detach()
        
        k=torch.cat((k_read, k), 1)
        v=torch.cat((v_read, v), 1)
        
        #shifted_k=[]
        #shifted_v=[]
        #for i in range(7): # 7 is d-1 for d=8
        #    shifted_k.append(torch.roll(k[:,:T//2],i,0))
        #    shifted_v.append(torch.roll(v[:,:T//2],i,0))
        #shifted_k=torch.stack(shifted_k).view(B,-1,C)
        #shifted_v=torch.stack(shifted_v).view(B,-1,C)
        
        #k=torch.cat((shifted_k, k), 1)
        #v=torch.cat((shifted_v, v), 1)
        
        q = q.view(B, T, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2) # (B, nh, T, hs)
        k_read = k_read.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2)
        v_read = v_read.view(B, -1, self.nhead, C // self.nhead).transpose(1, 2)
          
        
        # Causal Mask
        L = q.shape[2]
        S = k.shape[2]-q.shape[2]
        causal_mask = torch.ones(L, L, dtype=torch.bool, device='cuda').tril(diagonal=0)
        eye_mask=torch.eye(L, dtype=torch.bool, device='cuda')
        read_attnmask=torch.ones(L, L*3, dtype=torch.bool, device='cuda')
        aux=torch.arange(L).repeat_interleave(3)
        
        #new_attnmask=causal_mask[:,aux]
        read_attnmask=eye_mask[:,aux]
        
        attn_mask=torch.cat((read_attnmask,causal_mask),1)
        
        #shift_mask = torch.ones(L, int(L*3.5), dtype=torch.bool, device='cuda')
        #shift_mask[:T//2,:]=False
        #attn_mask=torch.cat((shift_mask,attn_mask),1)
        
        
        # Memory Mask
        memory_mask = torch.ones(L*3, L, dtype=torch.bool, device='cuda')
        memory_mask=torch.concat((torch.eye(L*3, dtype=torch.bool, device='cuda'), memory_mask),1)
        #memory_mask=torch.concat((~torch.ones(L*3, int(L*3.5), dtype=torch.bool, device='cuda'), memory_mask),1)
        
        # Associative Learning
        std=0.5
        noise=torch.randn_like(k_read)*std
        k_read=F.normalize(k_read)
        k_read=k_read+noise
        
        
        with torch.backends.cuda.sdp_kernel():
            y = F.scaled_dot_product_attention(q,k,v,attn_mask=attn_mask,
                                                dropout_p=self.dropout)
            v_read = F.scaled_dot_product_attention(k_read,k,v, attn_mask=memory_mask,
                                                    dropout_p=0)
            k_read = F.scaled_dot_product_attention(k_read,k,k, attn_mask=memory_mask,
                                                    dropout_p=0)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        k_read = k_read.transpose(1, 2).contiguous().view(B, T, -1)
        v_read = v_read.transpose(1, 2).contiguous().view(B, T, -1)
        
        # output projection
        y = self.resid_dropout(self.proj(y))
        return y, write_k, write_v, k_read, v_read
        #return y, write_k, write_v, None,None

    
class FFN(nn.Module):
    def __init__(self, d_model=512, dropout=0.1, bias=False, ffn_mult=4):
        super().__init__()
        self.fc    = nn.Linear(d_model, ffn_mult * d_model, bias=bias)
        self.gelu  = nn.GELU()
        self.proj  = nn.Linear(ffn_mult * d_model, d_model, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.gelu(x)
        x = self.proj(x)
        x = self.dropout(x)
        return x
    

class GPT_Block(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.0, bias=False, ffn_mult=4, seq_len=8):
        super().__init__()
        self.ln_1 = LayerNormNoBias(d_model, bias=bias)
        self.attn = Attention(d_model, nhead, bias, dropout, seq_len)
        self.ln_2 = LayerNormNoBias(d_model, bias=bias)
        self.mlp = FFN(d_model, dropout, bias, ffn_mult)

    def forward(self, x, is_causal=True):
        x_ln = self.ln_1(x)
        x = x + self.attn(x_ln, x_ln, x_ln, is_causal=is_causal)
        
        x = x + self.mlp(self.ln_2(x))
        
        return x

    def forward_xl_windowed(self, x, is_causal=True):
        x_ln = self.ln_1(x)
        x = x + self.attn.forward_xl_windowed(x_ln, x_ln, x_ln, is_causal=is_causal)
        
        x = x + self.mlp(self.ln_2(x))
        
        return x    
    


class GPT_Transformer(nsd_Module):
    def __init__(self, d_model, num_blks, nhead, seq_len,
                 dropout = 0.1, bias=False, report_params_count=True,
                 ffn_mult=4):
        super().__init__()

        #self.pos_encoding = nn.Sequential(nn.Linear(seq_len, d_model, bias=False),
        #                                  LayerNormNoBias(d_model)) #Stable Embedding Layer # Requires One Hot
        self.pos_encoding = nn.Embedding(seq_len, d_model)
        
        self.final_ln = LayerNormNoBias(d_model)
        self.start_dropout = nn.Dropout(dropout)

        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module("block"+str(i), GPT_Block(
                                d_model, nhead, dropout, bias=False, ffn_mult=ffn_mult, seq_len=seq_len))
            
        
        #nn.init.xavier_uniform_(self.pos_encoding[0].weight)
        
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_blks))
        
        if report_params_count:
            params_to_count = [p for p in self.parameters() if p.requires_grad]
            print(f'GPT Transformer Parameters: {sum(p.numel() for p in params_to_count)/1e6:.2f}M')
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
                
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
        elif isinstance(module, nn.LayerNorm):
            nn.init.constant_(module.bias, 0)
            nn.init.constant_(module.weight, 1.0)

        
    def forward(self, X, is_causal=True):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)[:X.shape[1]]
        X = self.start_dropout(X+pos_emb)

        for i, blk in enumerate(self.blks):
            X = blk(X, is_causal)
            
        return self.final_ln(X)

    def forward_xl_windowed(self, X, is_causal=True):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)[:X.shape[1]]
        X = self.start_dropout(X+pos_emb)

        for i, blk in enumerate(self.blks):
            X = blk.forward_xl_windowed(X, is_causal)
            
        return self.final_ln(X)    


class GPT_NLP(nsd_Module):
    def __init__(self, hiddens, num_blks, nhead, seq_len, vocab_size=50257,
                 temperature=1.0, k=20, p=0.9, sampling='gpt', report_params_count=True, tied_weights=True):
        super().__init__()
        
        
        self.emb_vocab = nn.Embedding(vocab_size, hiddens)
        self.gpt = GPT_Transformer(hiddens, nhead=nhead, num_blks=num_blks)
        
        self.cls = nn.Linear(hiddens, vocab_size, bias=False)
        
        if tied_weights:
            self.emb_vocab.weight = self.cls.weight

        
        if report_params_count:
            params_to_count = [p for p in self.parameters() if p.requires_grad]
            print(f'GPT NLP Parameters: {sum(p.numel() for p in params_to_count)/1e6:.2f}M')

    def forward(self, X, is_causal=True):
        batch_size, seq_len = X.shape
        
        mask = X>self.vocab_size
        X[mask] = self.vocab_size-1
        
        X = self.emb_vocab(X)
        #cls = torch.autograd.Variable(torch.zeros(batch_size, 2, self.hiddens)).to('cuda')
        
        #X = torch.cat((X, cls), dim=1)
        X = self.gpt(X, is_causal=is_causal)

        return self.cls(X)










class GPT_Block_XL(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.0, bias=False, ffn_mult=4, seq_len=8, windowed=False):
        super().__init__()
        self.ln_1 = LayerNormNoBias(d_model, bias=bias)
        if windowed:
            self.attn = Attention_XL_window(d_model, nhead, bias, dropout, seq_len=seq_len)
        else:
            self.attn = Attention_XL(d_model, nhead, bias, dropout)
        self.ln_2 = LayerNormNoBias(d_model, bias=bias)
        self.mlp = FFN(d_model, dropout, bias, ffn_mult)

    def forward(self, x, is_causal=True):
        x_ln = self.ln_1(x)
        x = x + self.attn(x_ln, x_ln, x_ln, is_causal=is_causal)
        
        x = x + self.mlp(self.ln_2(x))
        
        return x
    


class GPT_Transformer_XL(nsd_Module):
    def __init__(self, d_model, num_blks, nhead, seq_len,
                 dropout = 0.1, bias=False, report_params_count=True,
                 ffn_mult=4, windowed=False):
        super().__init__()

        #self.pos_encoding = nn.Sequential(nn.Linear(seq_len, d_model, bias=False),
        #                                  LayerNormNoBias(d_model)) #Stable Embedding Layer # Requires One Hot
        self.pos_encoding = nn.Embedding(seq_len, d_model)
        
        self.final_ln = LayerNormNoBias(d_model)
        self.start_dropout = nn.Dropout(dropout)

        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module("block"+str(i), GPT_Block_XL(
                                d_model, nhead, dropout, bias=False, ffn_mult=ffn_mult, seq_len=seq_len, windowed=windowed))
            
        
        #nn.init.xavier_uniform_(self.pos_encoding[0].weight)
        
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_blks))
        
        if report_params_count:
            params_to_count = [p for p in self.parameters() if p.requires_grad]
            print(f'GPT Transformer Parameters: {sum(p.numel() for p in params_to_count)/1e6:.2f}M')
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
                
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
        elif isinstance(module, nn.LayerNorm):
            nn.init.constant_(module.bias, 0)
            nn.init.constant_(module.weight, 1.0)

        
    def forward(self, X, is_causal=True):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)[:X.shape[1]]
        X = self.start_dropout(X+pos_emb)

        for i, blk in enumerate(self.blks):
            X = blk(X, is_causal)
            
        return self.final_ln(X)

















class Transformer_Block_NoLN(nsd_Module):
    def __init__(self, d_model, nhead, dropout=0.0, bias=False, ffn_mult=4, stochastic_depth=1):
        super().__init__()
        self.ln_1 = LayerNormNoBias(d_model, bias=bias)
        self.attn = Attention(d_model, nhead, bias, dropout)
        self.ln_2 = LayerNormNoBias(d_model, bias=bias)
        self.mlp = FFN(d_model, dropout, bias, ffn_mult)

    def forward(self, x, is_causal=True):
        #x = renormalize(x)
        keep_path = torch.ones(x.shape[0],device='cuda')*(self.stochastic_depth if self.training else 1)
        keep_path = torch.bernoulli(keep_path)[:,None,None]

        x_ln = self.ln_1(x)
        x = x + self.attn(x_ln, x_ln, x_ln, is_causal=is_causal)*keep_path
        
        x = x + self.mlp(self.ln_2(x))*keep_path
        
        return x


class Transformer_NoDATA(nn.Module):
    def __init__(self, d_model, num_blks, nhead, seq_len,
                 dropout = 0.1, bias=False, report_params_count=True,
                 ffn_mult=4, stochastic_depth=1.0, scale_init=1):
        super().__init__()
        self.num_hiddens = d_model
        self.scale_init=scale_init
        if scale_init==1:
            self.scale_init=num_blks


        self.pos_encoding = nn.Embedding(seq_len, d_model)

        self.final_ln = LayerNormNoBias(d_model)
        self.start_dropout = nn.Dropout(dropout)
        self.seq_len = seq_len
        self.num_blks=num_blks

        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module("block"+str(i), Transformer_Block_NoLN(
                                d_model, nhead, dropout, bias=False, ffn_mult=ffn_mult,
                                stochastic_depth=1-((1-stochastic_depth)*i/num_blks) ))


        # https://proceedings.mlr.press/v119/huang20f/huang20f.pdf

        self.apply(init_gpt)
        for pn, p in self.named_parameters():
            if pn.endswith('proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_blks))

    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            #torch.nn.init.normal_(module.weight, mean=0.0, std=1/math.sqrt(self.num_hiddens))
            torch.nn.init.xavier_uniform_(module.weight, gain=(torch.tensor(4*self.scale_init,dtype=torch.float)).pow(-1/4))
        
        

    def forward(self, X, is_causal=True):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)[:X.shape[1]]
        X = self.start_dropout(X+pos_emb)
        

        for i, blk in enumerate(self.blks):
            X = blk(X, is_causal)
            
        X = self.final_ln(X)
        
        return X
    
    def no_pos(self, X, is_causal=True):
        X = self.start_dropout(X)
        
        
        for i, blk in enumerate(self.blks):
            X = blk(X, is_causal)

        X = self.final_ln(X)
        
        return X
    
    def masked(self, X, mask, is_causal=True):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)[:X.shape[1]]
        X = self.start_dropout(X+pos_emb)
        X = X.gather(1, mask)
        
        
        for i, blk in enumerate(self.blks):
            X = blk(X, is_causal)

        X = self.final_ln(X)
        
        return X




    
def modulate(x, shift, scale):
    # x (B, T, D)
    # shift (B, D)
    # scale (B, D)
    
    return x * (1 + scale[:,None]) + shift[:,None]


class DiT_Block(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.0, bias=False, ffn_mult=4):
        super().__init__()
        self.ln_1 = LayerNormNoBias(d_model, bias=bias)
        self.attn = Attention(d_model, nhead, bias, dropout)
        self.ln_2 = LayerNormNoBias(d_model, bias=bias)
        self.mlp = FFN(d_model, dropout, bias, ffn_mult)
        
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(d_model, 6 * d_model, bias=True)
        )

        self.ln_1.apply(init_gpt)
        self.attn.apply(init_gpt)
        self.ln_2.apply(init_gpt)
        self.mlp.apply(init_gpt)
        self.adaLN_modulation.apply(init_zeros)
        
    def forward(self, x, c):
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation(c).chunk(6, dim=1)
        
        x_ln = modulate(self.ln_1(x), shift_msa, scale_msa)
        
        x = x + (1+gate_msa[:,None]) * self.attn(x_ln, x_ln, x_ln, is_causal=False)
        x = x + (1+gate_mlp[:,None]) * self.mlp(modulate(self.ln_2(x), shift_mlp, scale_mlp))
        
        return x

    def forward_no_dit(self, x):
        x_ln = self.ln_1(x)
        x = x + self.attn(x_ln, x_ln, x_ln, is_causal=False)
        return x + self.mlp(self.ln_2(x))
    
    
class DiT_Transformer(nsd_Module):
    def __init__(self, d_model, num_blks, nhead, seq_len,
                 dropout = 0.1, bias=False, report_params_count=True,
                 ffn_mult=4, scale_init=1):
        super().__init__()
        if scale_init==1:
            scale_init=num_blks

        self.pos_encoding = nn.Embedding(seq_len, d_model)
        
        self.final_ln = LayerNormNoBias(d_model)
        self.start_dropout = nn.Dropout(dropout)
        

        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module("block"+str(i), DiT_Block(
                                d_model, nhead, dropout, bias=False, ffn_mult=ffn_mult))
            
        
        #nn.init.xavier_uniform_(self.pos_encoding[0].weight)
        
        self.apply(init_gpt)
        self.init_weights()
        
        for pn, p in self.named_parameters():
            if pn.endswith('proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_blks))

        if report_params_count:
            params_to_count = [p for p in self.parameters() if p.requires_grad]
            print(f'GPT Transformer Parameters: {sum(p.numel() for p in params_to_count)/1e6:.2f}M')
    
    def init_weights(self):
        
        # Zero-out adaLN modulation layers in DiT blocks:
        for block in self.blks:
            block.adaLN_modulation[-1].apply(init_zeros)
    
        
    def forward(self, X, c):
        # Input:
        # X e (B, T, D)
        # c e (B, D)
        
        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)
        
        X = self.start_dropout(X+pos_emb)

        for i, blk in enumerate(self.blks):
            X = blk(X, c)
            
        return self.final_ln(X)
    

    def forward_no_dit(self, X):
        # Input:
        # X e (B, T, D)
        # c e (B, D)
        
        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)
        
        X = self.start_dropout(X+pos_emb)

        for i, blk in enumerate(self.blks):
            X = blk.forward_no_dit(X)
            
        return self.final_ln(X)
    
     
    

class CrossAttention_Block(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.0, bias=False):
        super().__init__()
        self.ln_1 = LayerNormNoBias(d_model, bias=bias)
        self.attn = Attention(d_model, nhead, bias, dropout)
        self.ln_2 = LayerNormNoBias(d_model, bias=bias)
        self.mlp = FFN(d_model, dropout, bias)

    def forward(self, q, k, v, is_causal=False):
        q = q + self.attn(self.ln_1(q),self.ln_1(k),self.ln_1(v), is_causal=is_causal)
        q = q + self.mlp(self.ln_2(q))
        return q
    


class CrossAttention_Transformer(nn.Module):
    def __init__(self, d_model, num_blks, nhead, seq_len, dim_feedforward=2048,  
                 dropout = 0.1, vocab_size = 0, bias=False):
        super().__init__()

        self.pos_encoding = nn.Embedding(seq_len, d_model)
        
        self.out_ln = LayerNormNoBias(d_model)
        self.start_dropout = nn.Dropout(dropout)
        self.seq_len = seq_len

        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module("block"+str(i), CrossAttention_Block(
                                d_model, nhead, dropout, bias=False))
            
        
        nn.init.xavier_uniform_(self.pos_encoding[0].weight)


        
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_blks))
        self.apply(self._init_weights)
        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            #torch.nn.init.xavier_normal_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            #torch.nn.init.xavier_normal_(module.weight)
    
    def forward(self, q, k, v, is_causal=False):

        pos = torch.arange(0, self.seq_len, dtype=torch.long, device='cuda')
        pos_emb = self.pos_encoding(pos)
        q = self.start_dropout(q+pos_emb)
        k = self.start_dropout(k+pos_emb)
        v = self.start_dropout(v+pos_emb)

        for i, blk in enumerate(self.blks):
            q = blk.forward(q,k,v, is_causal)
        q = self.out_ln(q)
        return q


    
    







class SpatialNorm(nn.Module):
    """
    Spatially conditioned normalization as defined in https://arxiv.org/abs/2209.09002.

    Args:
        f_channels (`int`):
            The number of channels for input to group normalization layer, and output of the spatial norm layer.
        zq_channels (`int`):
            The number of channels for the quantized vector as described in the paper.
    """

    def __init__(
        self,
        f_channels: int,
        zq_channels: int,
    ):
        super().__init__()
        self.norm_layer = nn.GroupNorm(num_channels=f_channels, num_groups=32, eps=1e-6, affine=True)
        self.conv_y = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)
        self.conv_b = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, f: torch.FloatTensor, zq: torch.FloatTensor) -> torch.FloatTensor:
        f_size = f.shape[-2:]
        zq = F.interpolate(zq, size=f_size, mode="nearest")
        norm_f = self.norm_layer(f)
        new_f = norm_f * self.conv_y(zq) + self.conv_b(zq)
        return new_f
    
    
    
class ConvAttnBlock(nn.Module):
    def __init__(self, in_channels, t_emb_dim=512, dropout=0, nhead=8):
        super().__init__()
        self.in_channels = in_channels
        self.dropout = dropout
        self.nhead = in_channels//nhead
        
        self.norm = nn.GroupNorm(32, in_channels)
        
        #self.norm = SpatialNorm(in_channels, t_emb_dim)

        self.q = torch.nn.Linear(in_channels,
                                 in_channels)
        self.k = torch.nn.Linear(in_channels,
                                 in_channels)
        self.v = torch.nn.Linear(in_channels,
                                 in_channels)
        self.proj_out = torch.nn.Linear(in_channels,
                                        in_channels)
        self.q.apply(init_cnn)
        self.k.apply(init_cnn)
        self.v.apply(init_cnn)
        self.proj_out.apply(init_cnn)


    def forward(self, x, t_emb=None):
        b, c, h, w = x.shape

        h_ = x
        h_ = self.norm(h_).view(b, c, h*w).transpose(1,2)
        
        #h_ = self.norm(h_, t_emb)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)
        q = q.contiguous().view(b, h*w, self.nhead, c//self.nhead).transpose(1, 2)
        k = k.contiguous().view(b, h*w, self.nhead, c//self.nhead).transpose(1, 2)
        v = k.contiguous().view(b, h*w, self.nhead, c//self.nhead).transpose(1, 2)

        # compute attention

        with torch.backends.cuda.sdp_kernel():
            h_ = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=False)

        h_ = h_.transpose(1, 2).view(b, h*w, c)
        h_ = self.proj_out(h_).transpose(1,2)

        h_ = h_.reshape(b, c, h, w)

        return x+h_

    """
    def forward(self, x, t_emb=None):
        h_ = x
        h_ = self.norm(h_)
        print(f"{h_.shape}")
        #h_ = self.norm(h_, t_emb)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b, c, h, w = q.shape
        q = q.view(b, c, h*w).transpose(1,2)
        k = k.view(b, c, h*w).transpose(1,2)
        v = v.view(b, c, h*w).transpose(1,2)
        '''
        w_ = torch.bmm(q, k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        w_ = w_.permute(0, 2, 1)   # b,hw,hw (first hw of k, second of q)
        # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = torch.bmm(v, w_)
        '''
        with torch.backends.cuda.sdp_kernel():
            h_ = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=False)

        h_ = h_.transpose(1, 2)
        h_ = h_.reshape(b, c, h, w)

        h_ = self.proj_out(h_)

        return x+h_
    """

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math




class MLP(nn.Module):
    def __init__(self, in_hiddens=512, med_hiddens=512, out_hiddens=512, layers=1,
                 init=init_relu, in_act=nn.SiLU(), out_act=nn.Identity(),
                 ln_eps=1e-3, last_init=init_xavier, bias=True):
        super().__init__()
        # Special MLP with custom options for non last layer and last layer Linears.

        modules=[]
        self.init=init
        self.last_init=last_init
        
        hiddens=in_hiddens
        _out_hiddens = med_hiddens
        act = in_act
        for l in range(layers):
            last_layer = l==(layers-1)
            if last_layer:
                _out_hiddens = out_hiddens
                act = out_act
            modules.append(nn.Linear(hiddens, _out_hiddens, bias=bias))
            
            modules.append(act)
            hiddens=med_hiddens
        self.mlp=nn.Sequential(*modules)
        #print(self.mlp)

        
        self.init_weights()

    def turn_off_grads(self):
        for layer in self.mlp:
            if hasattr(layer, 'weight'):
                layer.weight.requires_grad=False
            if hasattr(layer, 'bias'):
                layer.bias.requires_grad=False
    def init_weights(self):
        self.mlp.apply(self.init)
        self.mlp[-2].apply(self.last_init)
        
        
    def forward(self,X):
        return self.mlp(X)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


import numpy as np
import random




    
    
    
class ViT(nsd_Module):
    def __init__(self, d_model, num_blks, nhead, patches=(16,16), img_size=(96,72), first_channel=3,
                 dropout=0, bias=True, report_params_count=True,
                 ffn_mult=4, stochastic_depth=1.0):
        super().__init__()

        self.patches = np.prod(patches)
        self.N = int(np.prod(img_size) / self.patches)

        self.in_proj = MLP(first_channel * self.patches, out_hiddens=d_model, last_init=init_gpt)

        # Classe "token" de pooling
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.transformer = Transformer_NoDATA(
            d_model, num_blks, nhead, seq_len=self.N + 1,
            dropout=dropout, bias=bias, report_params_count=False,
            ffn_mult=ffn_mult, stochastic_depth=stochastic_depth
        )

        # Inicializar pesos do CLS token
        nn.init.normal_(self.cls_token, std=0.02)

        if report_params_count:
            params_count(self, 'ViT')

    def patchify(self, X):
        # Dividir a imagem em patches e reformatar
        X = X.view(-1, self.patches * self.first_channel, self.N).transpose(-2, -1)
        return X

    def proj(self, X):
        X = self.patchify(X)
        return self.in_proj(X)
    
    def transformers(self, X):
        
        X = self.transformer(X, is_causal=False).view(-1, self.stacked_frames*self.N, self.d_model)
        X = self.temporal_aggr(X, is_causal=False)
        
        return X[:,-self.N:]

    def masked(self, X, mask):
        
        X = self.transformer.masked(X, mask, is_causal=False).view(-1, self.stacked_frames*mask.shape[1], self.d_model)
        X = self.temporal_aggr(X, is_causal=False)
        
        return X[:,-mask.shape[1]:]
    
    def forward(self, X):
        # Criar patches e projetar
        X = self.patchify(X)
        X = self.in_proj(X)

        # Adicionar o token de classe
        batch_size = X.size(0)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        X = torch.cat((cls_tokens, X), dim=1)

        # Passar pelo transformer
        X = self.transformer(X, is_causal=False)

        # Retornar apenas o token de classe
        return X[:, 0]  # Forma: [batch_size, d_model]




class ViT_IWM(nsd_Module):
    def __init__(self, encoder,
                 d_predictor, num_blks_predictor, nhead_predictor,
                 stacked_frames=4,
                 mask_samples=4,
                 masked_tokens=4,
                 num_augmentations=3,
                 first_channel=3,
                 dropout = 0, bias=True, report_params_count=True,
                 ffn_mult=4, stochastic_depth=1.0):
        super().__init__()
        
        self.d_encoder = encoder.d_model
        
        
        self.first_channel = encoder.first_channel*stacked_frames
        self.img_size = encoder.img_size
        self.patches = encoder.patches
        self.N = encoder.N
        self.masked_tokens=self.N//masked_tokens

        # Mask
        self.mask = MLP(1, out_hiddens=d_predictor, last_init=init_xavier)
        self.mask_pos_encoding = nn.Embedding(self.N, d_predictor)
        self.mask_mlp = MLP(d_predictor+num_augmentations, d_predictor, d_predictor, layers=4, in_act=nn.ReLU(), out_act=nn.ReLU(),
                            init=init_relu, last_init=init_gpt)
        self.mask_pos_encoding.apply(init_gpt)

        # Encoder
        self.encoder = encoder

        # Predictor
        self.predictor_proj = MLP(self.d_encoder, out_hiddens=d_predictor, last_init=init_gpt) \
                              if d_predictor!=self.d_encoder else nn.Identity()

        self.predictor = Transformer_NoDATA(d_predictor, num_blks_predictor, nhead_predictor, seq_len=self.N+1,
                 dropout = dropout, bias=bias, report_params_count=False,
                 ffn_mult=ffn_mult, scale_init=num_blks_predictor, stochastic_depth=stochastic_depth)


        self.predictor_out_proj = MLP(d_predictor, out_hiddens=self.d_encoder, last_init=init_gpt) \
                              if d_predictor!=self.d_encoder else nn.Identity()

        if report_params_count:
            params_count(self, 'IWM')

    def hard_reset(self, new_network, alpha):
        network_ema(self.encoder, new_network.encoder, alpha)

        network_ema(self.predictor_proj, new_network.predictor_proj, alpha)
        network_ema(self.predictor, new_network.predictor, alpha)

        network_ema(self.mask, new_network.mask, alpha)
        network_ema(self.mask_pos_encoding, new_network.mask_pos_encoding, alpha)
        network_ema(self.mask_mlp, new_network.mask_mlp, alpha)

    def get_random_mask(self, X, augmentations):
        B, T, D = X.shape
        B = B//self.stacked_frames
        m_rand = self.mask_samples*random.randint(0,int(self.masked_tokens*2//self.mask_samples)-1)
        
        
        # Get non-overlapping mask
        mask_pos = torch.arange(T, device='cuda')[None,:].repeat_interleave(B,0).float()
        mask_pos = torch.multinomial(mask_pos, num_samples=self.masked_tokens+m_rand, replacement=False)
        
        mask_pos_repeat = mask_pos.repeat_interleave(self.stacked_frames,0)

        # Get the mask complement
        full_range = torch.arange(T,device='cuda')[None,:].repeat_interleave(B,0)

        complement = torch.zeros_like(full_range, dtype=torch.bool)
        complement.scatter_(1, mask_pos, 1)

        complement = full_range[~complement].view(mask_pos.shape[0], -1)
        

        # Mask mlp for geometric + augmentation informations
        mask = self.mask(torch.ones(B*self.stacked_frames,self.masked_tokens+m_rand,1, device='cuda'))

        mask = mask + self.mask_pos_encoding(mask_pos_repeat)

        augmentations = augmentations.repeat_interleave(self.stacked_frames,0)[:,None].expand(-1,mask.shape[1],-1)

        mask = self.mask_mlp(torch.cat((mask,augmentations),-1))

        # Expand to allow gather
        mask_pos = mask_pos[:,:,None].expand(-1,-1,X.shape[-1])
        complement = complement[:,:,None].expand(-1,-1,X.shape[-1])

        return X, mask_pos, complement, mask
        
    def patchify(self, X):
        X = X.view(-1, self.patches*self.first_channel, self.N).transpose(-2,-1)
        return X
    def get_block_mask(self, batch_size):
        
        all_wins = torch.zeros(self.first_channel,*self.img_size).long()
        
        b_mask, b_complement = [], []
        min_c_len = 999 # for trunked collate
        #min_m=999
        
        for b in range(batch_size):
            wins, complements = [], []
            for m in range(self.mask_samples):
                w,h = self.img_size


                min_ar, max_ar = (0.75, 1.5)
                aspect_ratio = min_ar + random.random() * (max_ar - min_ar)

                h_sample_size = int( (h*(torch.tensor(random.random())*0.05+0.15)) * aspect_ratio)

                w_wins, h_wins = torch.randint(0,h-h_sample_size,(2,)).split(1,0)
                win=all_wins.clone()


                for w_win, h_win in zip(w_wins, h_wins):
                    win[...,w_win:w_win+h_sample_size, h_win:h_win+h_sample_size]=1

                
                win = self.patchify(win.float()).mean(-1)
                
                values, idx = win.sort(descending=True)

                idx = idx[:,:self.N//4]
                
                #min_m = min(min_m, len(values[0].nonzero()))
                wins.append(idx)


            wins = torch.stack(wins).squeeze()


            full_range = torch.arange(win.shape[1])

            complement = torch.zeros_like(full_range, dtype=torch.bool)
            complement.scatter_(0, wins.view(-1).unique(), 1)

            complement = full_range[~complement]
            min_c_len = min(min_c_len, len(complement))
            
            
            b_mask.append(wins)
            b_complement.append(complement)
            
            
        for i in range(len(b_complement)):
            b_complement[i] = b_complement[i][:min_c_len]
        
        b_mask = torch.stack(b_mask).cuda()
        b_complement = torch.stack(b_complement).cuda()
        #print(min_m)
        
        return b_mask, b_complement
    
    def get_mask(self, X, augmentations):
        B = X.shape[0]//self.stacked_frames

        
        mask_pos, complement = self.get_block_mask(B)
        mask_pos = mask_pos.view(B*self.mask_samples,-1)
        
        
        
        mask = self.mask(torch.ones(B*self.mask_samples,1,1, device='cuda'))
        
        mask = mask + self.mask_pos_encoding(mask_pos)
        #augmentations = augmentations.repeat_interleave(self.stacked_frames*self.mask_samples,0)[:,None].expand(-1,mask.shape[1],-1)
        #mask = self.mask_mlp(torch.cat((mask,augmentations),-1))


        mask_pos = mask_pos[...,None].expand(-1,-1,self.d_encoder)
        complement = complement[...,None].expand(-1,-1,self.d_encoder).repeat_interleave(self.stacked_frames,0)
        
        return mask_pos, mask, complement
    
    def encode(self, X):
        return self.encoder(X)


    def forward(self, X, y, augmentations):
        X = self.encoder.proj(X)
        
        mask_pos, mask, complement = self.get_mask(X, augmentations)
        
        X = self.encoder.masked(X, complement)
        X = self.predictor_proj(X)

        X = torch.cat((X.repeat_interleave(4,0),mask),1)
        
        X = self.predictor.no_pos(X)[:,-mask.shape[1]:]
        X = self.predictor_out_proj(X)
        
        return X, y.repeat_interleave(4,0).gather(1,mask_pos)

In [5]:
def salvar_checkpoint_vit(modelo, classifier, otimizador, epoca, caminho):
    checkpoint = {
        'modelo': modelo.state_dict(),
        'classifier': classifier.state_dict(),
        'otimizador': otimizador.state_dict(),
        'epoca': epoca
    }
    torch.save(checkpoint, caminho)
    print(f"Checkpoint salvo: {caminho}")


# Função para carregar checkpoints
def carregar_checkpoint_vit(modelo, classifier, otimizador, caminho):
    if os.path.exists(caminho):
        checkpoint = torch.load(caminho)
        modelo.load_state_dict(checkpoint['modelo'])
        classifier.load_state_dict(checkpoint['classifier'])
        otimizador.load_state_dict(checkpoint['otimizador'])
        epoca_inicial = checkpoint['epoca']
        print(f"Checkpoint carregado: {caminho} (Época {epoca_inicial})")
        return epoca_inicial
    else:
        print(f"Nenhum checkpoint encontrado em: {caminho}. Começando do zero.")
        return 0


In [6]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Configuração do dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Configurações gerais
numero_de_epocas = 300
bs = 64
image_size = (96, 72)  # Atualizado para corresponder ao modelo customizado
patches = (16, 16)  # Tamanho do patch do ViT
num_classes = 7  # Atualize de acordo com seu dataset
checkpoint_dir = '../checkpoints/'

# Criar diretório de checkpoints se não existir
os.makedirs(checkpoint_dir, exist_ok=True)

# Transformações para as imagens
transformacoes_de_imagens = {
    'treino': transforms.Compose([
        transforms.Resize(size=image_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=image_size),
        transforms.ToTensor(),
    ])
}

# Carregar datasets
dataset = '../data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')

data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Criar DataLoaders
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True, num_workers=4)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=False, num_workers=4)

# Definir o modelo ViT customizado
 

vit_model = ViT(
    d_model=512,  # Dimensão do modelo
    num_blks=12,  # Número de blocos do transformer
    nhead=8,  # Número de cabeças de atenção
    patches=patches,
    img_size=image_size,
    first_channel=3,  # Número de canais de entrada
    dropout=0.1,
    report_params_count=True
)

# Classificador para ajustar a saída do ViT
classifier = nn.Sequential(
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

vit_model.to(device)
classifier.to(device)

# Definir a função de erro e o otimizador
funcao_erro = nn.CrossEntropyLoss()  # Negative Log Likelihood Loss
otimizador = optim.AdamW(
    list(vit_model.parameters()) + list(classifier.parameters()), 
    lr=0.0001
)

checkpoint_path = os.path.join(checkpoint_dir, 'ultimo_checkpoint_nosavedata01.pth')
epoca_inicial = carregar_checkpoint_vit(vit_model, classifier, otimizador, checkpoint_path)

# Atualizar a função `treinar_e_validar` para salvar sempre no mesmo arquivo o último checkpoint
def treinar_e_validar(modelo, classifier, metrica_erro, otimizador_sgd, epocas=25, iniciar_epoca=0, melhor_acuracia=0.0):
    scaler = torch.cuda.amp.GradScaler()  # Inicializar GradScaler para Mixed Precision
    historico = []

    for epoca in range(iniciar_epoca, epocas):
        inicio_epoca = time.time()
        print(f"\nÉpoca {epoca + 1}/{epocas}")

        # Modo de treinamento
        modelo.train()
        classifier.train()
        erro_treino = 0.0
        acuracia_treino = 0.0

        for entradas, labels in tqdm(data_loader_treino, desc="Treinando"):
            entradas, labels = entradas.to(device), labels.to(device)
            otimizador_sgd.zero_grad()

            # Forward pass
            with torch.cuda.amp.autocast():  # Mixed Precision
                features = modelo(entradas)  # Extrair features do ViT
                saidas = classifier(features)  # Passar pelo classificador
                erro = metrica_erro(saidas, labels)  # Calcular perda

            # Backward pass
            scaler.scale(erro).backward()
            scaler.step(otimizador_sgd)
            scaler.update()

            erro_treino += erro.item() * entradas.size(0)
            _, preds = torch.max(saidas, 1)
            acuracia_treino += torch.sum(preds == labels.data)

        # Modo de avaliação
        modelo.eval()
        classifier.eval()
        erro_validacao = 0.0
        acuracia_validacao = 0.0

        # Inicializar variáveis para calcular a acurácia por classe
        total_por_classe = torch.zeros(num_classes, device=device)
        corretos_por_classe = torch.zeros(num_classes, device=device)

        with torch.no_grad():
            for entradas, labels in tqdm(data_loader_validacao, desc="Validando"):
                entradas, labels = entradas.to(device), labels.to(device)
                with torch.cuda.amp.autocast():
                    features = modelo(entradas)
                    saidas = classifier(features)
                    erro = metrica_erro(saidas, labels)

                erro_validacao += erro.item() * entradas.size(0)
                _, preds = torch.max(saidas, 1)
                acuracia_validacao += torch.sum(preds == labels.data)

                # Atualizar contadores por classe
                for classe in range(num_classes):
                    total_por_classe[classe] += torch.sum(labels == classe)
                    corretos_por_classe[classe] += torch.sum((preds == classe) & (labels == classe))

        # Calcular métricas
        erro_medio_treino = erro_treino / len(data['treino'])
        acuracia_medio_treino = acuracia_treino.double() / len(data['treino'])
        erro_medio_validacao = erro_validacao / len(data['validacao'])
        acuracia_medio_validacao = acuracia_validacao.double() / len(data['validacao'])

        historico.append([erro_medio_treino, erro_medio_validacao, acuracia_medio_treino, acuracia_medio_validacao])

        print(f"Treino - Erro: {erro_medio_treino:.4f}, Acurácia: {acuracia_medio_treino:.4f}")
        print(f"Validação - Erro: {erro_medio_validacao:.4f}, Acurácia: {acuracia_medio_validacao:.4f}")

        # Mostrar acurácia por classe
        print("Acurácia por classe na validação:")
        for classe in range(num_classes):
            taxa_acerto = (corretos_por_classe[classe] / total_por_classe[classe]).item() if total_por_classe[classe] > 0 else 0.0
            print(f"Classe {classe}: {taxa_acerto * 100:.2f}%")

        # Salvar checkpoints
        salvar_checkpoint_vit(modelo, classifier, otimizador_sgd, epoca + 1, checkpoint_path)

        # Atualizar o melhor modelo
        if acuracia_medio_validacao > melhor_acuracia:
            melhor_acuracia = acuracia_medio_validacao
            torch.save(modelo.state_dict(), 'melhor_modelo_nosavedata01.pth')
            torch.save(classifier.state_dict(), 'melhor_classifier_nosavedata01.pth')
            print("Melhor modelo salvo!")

    return historico


# Treinar o modelo
historico = treinar_e_validar(vit_model, classifier, funcao_erro, otimizador, numero_de_epocas, iniciar_epoca=epoca_inicial)

Usando o dispositivo: cuda:0
ViT Parameters: 38.17M


C:\Users\Danil\AppData\Local\Temp\ipykernel_15480\1993890487.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(caminho)
C:\Users\Danil\AppData\Loc

Checkpoint carregado: ../checkpoints/ultimo_checkpoint_nosavedata01.pth (Época 193)

Época 194/300


Treinando:   0%|          | 0/449 [00:00<?, ?it/s]C:\Users\Danil\AppData\Local\Temp\ipykernel_15480\2411256981.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed Precision
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
C:\Users\Danil\AppData\Local\Temp\ipykernel_15480\2782951431.py:168: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropou

Treino - Erro: 0.9456, Acurácia: 0.6469
Validação - Erro: 1.8062, Acurácia: 0.4558
Acurácia por classe na validação:
Classe 0: 27.62%
Classe 1: 30.36%
Classe 2: 32.66%
Classe 3: 58.77%
Classe 4: 41.35%
Classe 5: 43.64%
Classe 6: 64.10%
Checkpoint salvo: ../checkpoints/ultimo_checkpoint_nosavedata01.pth
Melhor modelo salvo!

Época 195/300


...

Treino - Erro: 0.5903, Acurácia: 0.7842
Validação - Erro: 2.4369, Acurácia: 0.4745
Acurácia por classe na validação:
Classe 0: 37.47%
Classe 1: 50.00%
Classe 2: 35.48%
Classe 3: 64.47%
Classe 4: 46.46%
Classe 5: 28.33%
Classe 6: 67.47%
Checkpoint salvo: ../checkpoints/ultimo_checkpoint_nosavedata01.pth

Época 299/300


Validando: 100%|██████████| 57/57 [00:30<00:00,  1.84it/s]


Treino - Erro: 0.5886, Acurácia: 0.7819
Validação - Erro: 2.4249, Acurácia: 0.4689
Acurácia por classe na validação:
Classe 0: 33.40%
Classe 1: 41.07%
Classe 2: 37.30%
Classe 3: 61.56%
Classe 4: 40.03%
Classe 5: 38.59%
Classe 6: 65.78%
Checkpoint salvo: ../checkpoints/ultimo_checkpoint_nosavedata01.pth

Época 300/300


Validando: 100%|██████████| 57/57 [00:30<00:00,  1.86it/s]


Treino - Erro: 0.5937, Acurácia: 0.7840
Validação - Erro: 2.5008, Acurácia: 0.4717
Acurácia por classe na validação:
Classe 0: 35.33%
Classe 1: 44.64%
Classe 2: 37.10%
Classe 3: 59.66%
Classe 4: 39.37%
Classe 5: 40.12%
Classe 6: 68.43%
Checkpoint salvo: ../checkpoints/ultimo_checkpoint_nosavedata01.pth


In [7]:
import matplotlib.pyplot as plt
import numpy as np

# Separar os dados do histórico
historico = np.array(historico)
erro_treino = historico[:, 0]
erro_validacao = historico[:, 1]
acuracia_treino = historico[:, 2]
acuracia_validacao = historico[:, 3]

# Configurar as épocas
epocas = range(1, len(erro_treino) + 1)

# Plotar gráfico de erro
plt.figure(figsize=(10, 5))
plt.plot(epocas, erro_treino, label='Erro - Treino')
plt.plot(epocas, erro_validacao, label='Erro - Validação')
plt.title('Erro por Época')
plt.xlabel('Época')
plt.ylabel('Erro')
plt.legend()
plt.grid()
plt.show()

# Plotar gráfico de acurácia
plt.figure(figsize=(10, 5))
plt.plot(epocas, acuracia_treino, label='Acurácia - Treino')
plt.plot(epocas, acuracia_validacao, label='Acurácia - Validação')
plt.title('Acurácia por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid()
plt.show()


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.